## TF ver

In [1]:
import tensorflow as tf
import numpy as np
import csv

### Data reading

In [2]:
# data reading
with open('./data/titanicTrain.csv', 'r') as f:
    data = list(csv.reader((f)))
    data = np.array(data)

In [3]:
data = data[:1000]
data[0, [0, 3, 4, 5, 6, 8]]

array(['pclass', 'sex', 'parch', 'fare'], 
      dtype='<U82')

In [4]:
data

array([['pclass', 'survived', 'name', ..., 'boat', 'body', 'home.dest'],
       ['1', '1', 'Allen, Miss. Elisabeth Walton', ..., '2', '',
        'St Louis, MO'],
       ['1', '1', 'Allison, Master. Hudson Trevor', ..., '11', '',
        'Montreal, PQ / Chesterville, ON'],
       ..., 
       ['3', '0', 'Markun, Mr. Johann', ..., '', '', ''],
       ['3', '1', 'Masselmani, Mrs. Fatima', ..., 'C', '', ''],
       ['3', '0', 'Matinoff, Mr. Nicola', ..., '', '', '']], 
      dtype='<U82')

In [8]:
def data_clean(data, withLabel=True):
    x_train = data[1:, [0, 3, 4, 5, 6, 8]]
    y_label = data[1:, 1]
    y_train = list()
    pclass = list()
    sex = list()
    embarked = list()
    for i in range(len(x_train)):
        # sex -- class data one hot
        if x_train[i, 1] == 'male':
            sex.append([5, 0])
        else:
            sex.append([0, 5])
        # age
        if x_train[i, 2] == '':
            x_train[i, 2] = '-100'
        # fare
        if x_train[i, 5] == '':
            x_train[i, 5] = '-200'
        # pclass -- class data one hot
        if x_train[i, 0] == '1':
            pclass.append([1, 0, 0])
        elif x_train[i, 0] == '2':
            pclass.append([0, 1, 0])
        else:
            pclass.append([0, 0, 1])
        # labels one hot
        if y_label[i] == '0':
            y_train.append([1, 0])
        else:
            y_train.append([0, 1])
    y_train = np.array(y_train)
    sex = np.array(sex)
    pclass = np.array(pclass)
    x_train = x_train[:, [2, 3, 4, 5]].astype('float32')
    # normalization
    x_train[:, 0] = x_train[:, 0]/100
    x_train[:, 1] = x_train[:, 1]/6
    x_train[:, 2] = x_train[:, 2]/6
    x_train[:, 3] = x_train[:, 2]/200
    # combination of class-data and continuous data
    x_train = np.hstack([pclass, sex, x_train])
    if withLabel: 
        return x_train, y_train
    else:
        return x_train

x_train, y_train = data_clean(data)
x_train.shape, y_train.shape

((999, 9), (999, 2))

In [9]:
# random splitting of training set and testing set
ind = np.arange(len(x_train))
np.random.shuffle(ind)
ind_train = ind[0:int(len(x_train)*0.8)+1]
ind_test = ind[int(len(x_train)*0.8)+1:]
x_test = x_train[ind_test]
y_test = y_train[ind_test]
x_train = x_train[ind_train]
y_train = y_train[ind_train]
len(x_test), len(x_train)

(199, 800)

In [10]:
# validation data
with open('./data/titanicQuestion.csv', 'r') as f:
    data_valid = list(csv.reader((f)))
    data_valid = np.array(data_valid)

x_valid = data_clean(data_valid, withLabel=False)
x_valid.shape

(309, 9)

In [11]:
from collections import Counter
Counter(data[1:, 1])

Counter({'0': 577, '1': 422})

### hyperparameter

In [12]:
epoch = 2000
l_r = 1e-4
n_in = len(x_train[0])
n_out = len(y_train[0])
neuron_number = [n_in, 128, 128, n_out]
keep_prob = 0.8

### placeholder

In [13]:
x = tf.placeholder(tf.float32, [None, n_in])
y = tf.placeholder(tf.float32, [None, n_out])

### network structure

In [14]:
def _weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def _bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [15]:
def _add_layer(x, n1, n2):
    # create weight and bias by the previous neuron number and next neuron number
    w = _weight([n1, n2])
    b = _bias([n2])
    l = tf.add(tf.matmul(x, w), b, name='nn')
    return l, w, b

def DNN(neuron_number, x, hidden_act=tf.nn.relu, isLog=False, out_act=tf.nn.softmax, keep_prob=.9):
    # neuron_number: array of neuron numbers for each layer, include input layer and output layer
    weights = dict()
    biases = dict()
    layers = dict()
    layers[0] = x
    # create layers
    for n in range(len(neuron_number)-1):
        layers[n+1], weights[n+1], biases[n+1] = _add_layer(layers[n],
                                                            neuron_number[n],
                                                            neuron_number[n+1])
        if n == (len(neuron_number)-2):
            if out_act == None:
                pass
            else:
                layers[n+1] = out_act(layers[n+1])
        else:
            if hidden_act == None:
                pass
            else:
                layers[n+1] = hidden_act(layers[n+1])
            layers[n+1] = tf.nn.dropout(layers[n+1], keep_prob=keep_prob)
    return layers, weights, biases

In [16]:
layers, weights, biases = DNN(neuron_number=neuron_number, 
                              x=x,
                              hidden_act=tf.nn.relu,
                              out_act=tf.nn.softmax,
                              keep_prob=keep_prob)
pred = layers[len(neuron_number)-1]
layers

{0: <tf.Tensor 'Placeholder:0' shape=(?, 9) dtype=float32>,
 1: <tf.Tensor 'dropout/mul:0' shape=(?, 128) dtype=float32>,
 2: <tf.Tensor 'dropout_1/mul:0' shape=(?, 128) dtype=float32>,
 3: <tf.Tensor 'Softmax:0' shape=(?, 2) dtype=float32>}

### loss func, accu and op

In [17]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
accu = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), tf.float32))
op = tf.train.RMSPropOptimizer(l_r).minimize(loss)

### Training

In [18]:
import time
init = tf.global_variables_initializer()
np.set_printoptions(precision=3)
batch_size = 800
with tf.Session() as sess:
    sess.run(init)
    for epo in range(epoch):
        for i in range(int(len(x_train)/batch_size)):
            sess.run(op, feed_dict={
                x:x_train[i*batch_size:(i+1)*batch_size],
                y:y_train[i*batch_size:(i+1)*batch_size]
            })
        if (epo+1)%100 == 0:
            loss_train, accu_train = sess.run([loss, accu], feed_dict={
                x:x_train,
                y:y_train
            })
            loss_test, accu_test = sess.run([loss, accu], feed_dict={
                x:x_test,
                y:y_test
            })
            loss_train = np.array([loss_train])
            accu_train = np.array([accu_train])
            loss_test = np.array([loss_test])
            accu_test = np.array([accu_test])
            print('Epoch:', epo+1,
                  ', TrainLoss:', loss_train,
                  ', TrainAccu:', accu_train,
                  ', TestLoss:', loss_test,
                  ', TestAccu:', accu_test)
    pred_q = sess.run(pred, feed_dict={
        x:x_valid
    })

Epoch: 100 , TrainLoss: [ 0.617] , TrainAccu: [ 0.775] , TestLoss: [ 0.626] , TestAccu: [ 0.749]
Epoch: 200 , TrainLoss: [ 0.534] , TrainAccu: [ 0.791] , TestLoss: [ 0.554] , TestAccu: [ 0.764]
Epoch: 300 , TrainLoss: [ 0.523] , TrainAccu: [ 0.791] , TestLoss: [ 0.549] , TestAccu: [ 0.764]
Epoch: 400 , TrainLoss: [ 0.513] , TrainAccu: [ 0.795] , TestLoss: [ 0.536] , TestAccu: [ 0.769]
Epoch: 500 , TrainLoss: [ 0.51] , TrainAccu: [ 0.79] , TestLoss: [ 0.53] , TestAccu: [ 0.789]
Epoch: 600 , TrainLoss: [ 0.508] , TrainAccu: [ 0.791] , TestLoss: [ 0.533] , TestAccu: [ 0.759]
Epoch: 700 , TrainLoss: [ 0.506] , TrainAccu: [ 0.791] , TestLoss: [ 0.533] , TestAccu: [ 0.784]
Epoch: 800 , TrainLoss: [ 0.5] , TrainAccu: [ 0.805] , TestLoss: [ 0.538] , TestAccu: [ 0.764]
Epoch: 900 , TrainLoss: [ 0.5] , TrainAccu: [ 0.811] , TestLoss: [ 0.537] , TestAccu: [ 0.774]
Epoch: 1000 , TrainLoss: [ 0.504] , TrainAccu: [ 0.798] , TestLoss: [ 0.532] , TestAccu: [ 0.769]
Epoch: 1100 , TrainLoss: [ 0.498] , 

In [63]:
sur = [np.argmax(pred_q[i]) for i in range(len(pred_q))]
Counter(sur), sur
# 1:  258, 51
# 2:  252, 57

(Counter({0: 260, 1: 49}),
 [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [47]:
with open('pred_result_tfver.csv', 'w') as f:
    for i in range(len(sur)):
        f.write(str(sur[i])+', \n')

In [25]:
pred_q

array([[  1.000e+00,   1.136e-10],
       [  9.989e-01,   1.113e-03],
       [  9.958e-01,   4.196e-03],
       [  1.000e+00,   2.761e-12],
       [  2.604e-03,   9.974e-01],
       [  1.000e+00,   1.772e-10],
       [  3.793e-03,   9.962e-01],
       [  9.083e-02,   9.092e-01],
       [  9.555e-01,   4.449e-02],
       [  1.000e+00,   1.835e-09],
       [  1.000e+00,   3.027e-08],
       [  9.313e-01,   6.873e-02],
       [  3.819e-02,   9.618e-01],
       [  1.401e-03,   9.986e-01],
       [  1.049e-02,   9.895e-01],
       [  1.000e+00,   1.903e-12],
       [  1.000e+00,   4.642e-11],
       [  1.000e+00,   1.686e-07],
       [  1.000e+00,   3.219e-11],
       [  1.000e+00,   5.033e-08],
       [  1.000e+00,   1.180e-07],
       [  1.000e+00,   2.468e-07],
       [  1.000e+00,   9.915e-12],
       [  5.197e-03,   9.948e-01],
       [  1.000e+00,   9.170e-09],
       [  9.999e-01,   1.317e-04],
       [  1.187e-01,   8.813e-01],
       [  1.000e+00,   2.305e-10],
       [  7.855e-01,

## tf.one_hot example

In [31]:
with tf.Session() as sess:
    one_hot = sess.run(tf.one_hot(y_train, depth=2))
one_hot

array([[[ 0.,  1.]],

       [[ 0.,  1.]],

       [[ 1.,  0.]],

       ..., 
       [[ 1.,  0.]],

       [[ 0.,  1.]],

       [[ 1.,  0.]]], dtype=float32)

In [32]:
one_hot.shape

(999, 1, 2)

In [56]:
x_train[:, 0][1]

1.0